<a href="https://colab.research.google.com/github/ShreyasheeSinha/Determining-Robustness-of-NLU-Models/blob/main/analysis_notebooks/Final_test_set_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The sheet which generates the final test dataset in the format discussed in the meetings.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
def read_jsonl_file(path):
  with open(path, 'r') as json_file:
      json_list = list(json_file)

  results = []
  for json_str in json_list:
      results.append(json.loads(json_str))
  return results

In [ ]:
test_data = pd.DataFrame(read_jsonl_file("/content/gdrive/MyDrive/paraphraser_work/Final_datasets/rte_test_p.jsonl")) # Unannotated dataset: /content/gdrive/MyDrive/paraphraser_work/MTurk data/Generated_dataset/rte_test_paraphrased_p_d_or_h_d_sorted.jsonl
test_data = test_data.fillna(value=np.nan)

In [ ]:
test_data

,dataset,corpus_sent_id,gold_label,task,sentence1,sentence2,s2_para_id,silver_label,s1_para_id
0,RTE1_test.column,25,False,IR,"The film was the evening 's big winner , baggi...",Two films won six Oscars .,NaN,NaN,NaN
1,RTE1_test.column,25,False,IR,"The film was the big winner of the evening, wi...",Two films won six Oscars .,NaN,False,RTE1_test.column_s1_25_para1
2,RTE1_test.column,49,True,IR,Kidnappings in Argentina have increased more t...,Argentina sees upsurge in kidnappings .,NaN,NaN,NaN
3,RTE1_test.column,49,True,IR,Kidnappings in Argentina have increased more t...,Argentine sees upsurge in kidnappings.,RTE1_test.column_s2_49_para1,True,NaN
4,RTE1_test.column,75,True,IR,"At the close , the Dow Jones industrial averag...",Dow Jones is down .,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2035,RTE3_test.column,796,True,SUM,It has pledged to allow the yuan to float more...,China has an export-driven economy.,RTE3_test.column_s2_796_para1,True,NaN
2036,RTE3_test.column,798,False,SUM,"The world 's largest consumer of oil , US impo...",There was a conflict between Israel and Lebanon .,NaN,NaN,NaN
2037,RTE3_test.column,798,False,SUM,"The world's largest consumer of oil, US import...",There was a conflict between Israel and Lebanon .,NaN,False,RTE3_test.column_s1_798_para1
2038,RTE3_test.column,800,False,SUM,It 's very difficult to get teams from China t...,It is difficult to get the right to stay in Ch...,NaN,NaN,NaN


In [ ]:
def generate_formatted_data(df):
  grouped_df = df.groupby(["dataset", "corpus_sent_id"])
  incorrect_preds_df = pd.DataFrame()
  index = 0
  for name, group in grouped_df:
    row_dict = {}
    ph = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]
    paraphrased = group[~(group['s1_para_id'].isnull() & group['s2_para_id'].isnull())]
    entailment_dict = {0: "not_entailed", 1: "entailed"}
    for index, row in paraphrased.iterrows():
      row = row.fillna(value="")
      row_dict["id"] = "prte_" + str(index)
      row_dict["original_pair_premise"] = ph.iloc[0]['sentence1']
      row_dict["original_pair_hypothesis"] = ph.iloc[0]['sentence2']
      row_dict["gold_label"] = entailment_dict[ph.iloc[0]['gold_label']]
      row_dict["original_pair_id"] = ph.iloc[0]['dataset'] + "_" + ph.iloc[0]['corpus_sent_id'].astype(str) + "_" + ph.iloc[0]['task']
      row_dict["paraphrased_pair_premise"] = row['sentence1']
      row_dict["paraphrased_pair_hypothesis"] = row['sentence2']
      paraphrase_id = row_dict["original_pair_id"]
      # print(row["s1_para_id"], row["s2_para_id"])
      if row["s1_para_id"] != "" and row["s2_para_id"] != "":
        paraphrase_id += "_ph"
      elif row["s1_para_id"] == "":
        paraphrase_id += "_h"
      else:
        paraphrase_id += "_p"
      row_dict["paraphrased_pair_id"] = paraphrase_id
      incorrect_preds_df = incorrect_preds_df.append(row_dict, ignore_index=True)
      index += 1
      # break
      # print(row_dict)
      # break
    # break
  return incorrect_preds_df

In [ ]:
test_data_final = generate_formatted_data(test_data)

In [ ]:
test_data_final

,id,original_pair_premise,original_pair_hypothesis,gold_label,original_pair_id,paraphrased_pair_premise,paraphrased_pair_hypothesis,paraphrased_pair_id
0,prte_1,"The film was the evening 's big winner , baggi...",Two films won six Oscars .,not_entailed,RTE1_test.column_25_IR,"The film was the big winner of the evening, wi...",Two films won six Oscars .,RTE1_test.column_25_IR_p
1,prte_3,Kidnappings in Argentina have increased more t...,Argentina sees upsurge in kidnappings .,entailed,RTE1_test.column_49_IR,Kidnappings in Argentina have increased more t...,Argentine sees upsurge in kidnappings.,RTE1_test.column_49_IR_h
2,prte_5,"At the close , the Dow Jones industrial averag...",Dow Jones is down .,entailed,RTE1_test.column_75_IR,"At the close , the Dow Jones industrial averag...",Dow Jones has fallen.,RTE1_test.column_75_IR_h
3,prte_7,The Los Angeles Lakers won their 3 rd straight...,Lakers win the championship .,entailed,RTE1_test.column_82_IR,The Los Angeles Lakers won their 3 rd straight...,Lakers won the championship.,RTE1_test.column_82_IR_h
4,prte_8,The Los Angeles Lakers won their 3 rd straight...,Lakers win the championship .,entailed,RTE1_test.column_82_IR,The Los Angeles Lakers won their 3rd straight ...,Lakers win the championship .,RTE1_test.column_82_IR_p
...,...,...,...,...,...,...,...,...
1121,prte_2031,"In Moscow , Andrei Kokoshin , the former Russi...",Andrei Kokoshin is the former Russian Security...,entailed,RTE3_test.column_787_SUM,"In Moscow , Andrei Kokoshin , the former Russi...",Andrei Kokoshin is the former head of the Russ...,RTE3_test.column_787_SUM_h
1122,prte_2033,The U.S. will seek to field a limited number o...,The U.S. buys ground-based defense silo launch...,not_entailed,RTE3_test.column_788_SUM,The U.S. will seek to field a limited number o...,The US purchases ground-based defense silo lau...,RTE3_test.column_788_SUM_h
1123,prte_2035,It has pledged to allow the yuan to float more...,China has an export-led economy .,entailed,RTE3_test.column_796_SUM,It has pledged to allow the yuan to float more...,China has an export-driven economy.,RTE3_test.column_796_SUM_h
1124,prte_2037,"The world 's largest consumer of oil , US impo...",There was a conflict between Israel and Lebanon .,not_entailed,RTE3_test.column_798_SUM,"The world's largest consumer of oil, US import...",There was a conflict between Israel and Lebanon .,RTE3_test.column_798_SUM_p


In [ ]:
test_data_final.to_json("/content/gdrive/MyDrive/paraphraser_work/Final_datasets/prte_test.json")
test_data_final.to_csv("/content/gdrive/MyDrive/paraphraser_work/Final_datasets/prte_test.csv", index=False)